In [2]:
# Setting up bell states and n-bit equivalents

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

In [3]:
n = int(input("How many qubits do you want to be entangled to the first?"))
total_qubits = n+1
total_bits = total_qubits

simulator = AerSimulator()
circuit = QuantumCircuit(total_qubits,total_bits)

circuit.h(0)
for i in range(1,total_qubits):
    circuit.cnot(0,i)

circuit.barrier() # stop measurements being moved and making it ugly
for i in range(total_qubits):
    circuit.measure(i,i)

print(circuit.draw())

compiled_circuit = transpile(circuit, simulator)

job = simulator.run(compiled_circuit, shots=5, memory=True)
result = job.result()
outcome = result.get_memory()
print(outcome)

     ┌───┐           ░ ┌─┐      
q_0: ┤ H ├──■────■───░─┤M├──────
     └───┘┌─┴─┐  │   ░ └╥┘┌─┐   
q_1: ─────┤ X ├──┼───░──╫─┤M├───
          └───┘┌─┴─┐ ░  ║ └╥┘┌─┐
q_2: ──────────┤ X ├─░──╫──╫─┤M├
               └───┘ ░  ║  ║ └╥┘
c: 3/═══════════════════╩══╩══╩═
                        0  1  2 
['111', '000', '111', '000', '111']


As expected, once the first qubit has been measured, all others will be measured in the same state.

Now we'll show that measurement of any of the bits dictates future measurements of the others.

In [4]:
n = int(input("How many qubits do you want to be entangled to the first?"))
total_qubits = n+1
total_bits = total_qubits

simulator = AerSimulator()
circuit = QuantumCircuit(total_qubits,total_bits)

circuit.h(0)
for i in range(1,total_qubits):
    circuit.cnot(0,i)

first_measure_index = 1
circuit.measure(first_measure_index,first_measure_index)

circuit.barrier() # stop measurements being moved and making it ugly
for i in [ix for ix in range(total_qubits) if ix != first_measure_index]:
    circuit.measure(i,i)

print(circuit.draw())

compiled_circuit = transpile(circuit, simulator)

job = simulator.run(compiled_circuit, shots=5, memory=True)
result = job.result()
outcome = result.get_memory()
print(outcome)

     ┌───┐                        ░ ┌─┐         
q_0: ┤ H ├──■────■───────■────■───░─┤M├─────────
     └───┘┌─┴─┐  │  ┌─┐  │    │   ░ └╥┘         
q_1: ─────┤ X ├──┼──┤M├──┼────┼───░──╫──────────
          └───┘┌─┴─┐└╥┘  │    │   ░  ║ ┌─┐      
q_2: ──────────┤ X ├─╫───┼────┼───░──╫─┤M├──────
               └───┘ ║ ┌─┴─┐  │   ░  ║ └╥┘┌─┐   
q_3: ────────────────╫─┤ X ├──┼───░──╫──╫─┤M├───
                     ║ └───┘┌─┴─┐ ░  ║  ║ └╥┘┌─┐
q_4: ────────────────╫──────┤ X ├─░──╫──╫──╫─┤M├
                     ║      └───┘ ░  ║  ║  ║ └╥┘
c: 5/════════════════╩═══════════════╩══╩══╩══╩═
                     1               0  2  3  4 
['11111', '11111', '11111', '00000', '11111']


Now we'll entangle based on a sequence.

In [29]:
seq = input("What sequence would you like to entangle with qubit 0?")
was_binary = True
# we're expecting the sequence to be in binary
try:
    bin = int(seq,base=2)
except ValueError:
    if seq.isalnum():
        new_seq = ""
        for c in seq:
            new_seq += f"{ord(c):08b}"
        print(new_seq)
        seq = new_seq
        was_binary = False
n = len(seq)

zeroes = [i+1 for i,e in enumerate(seq) if e=="0"]
ones = [i+1 for i,e in enumerate(seq) if e=="1"]

total_qubits = n+1
total_bits = total_qubits

simulator = AerSimulator()
circuit = QuantumCircuit(total_qubits,total_bits)

circuit.h(0)
for i in zeroes:
    circuit.cnot(0,i)

circuit.barrier() # stop measurements being moved and making it ugly
circuit.x(0)    # X gate so now we are entangling for 1 if the q0 state is 0
for i in ones:
    circuit.cnot(0,i)

circuit.barrier()
for i in range(total_qubits):
    circuit.measure(i,total_bits-1-i)

#print(circuit.draw())

compiled_circuit = transpile(circuit, simulator)

job = simulator.run(compiled_circuit, shots=5, memory=True)
result = job.result()
outcomes = result.get_memory()

if not was_binary:
    for f_outcome in outcomes:
        outcome = f_outcome[1:]
        ascii_bytes = [outcome[i:i+8] for i in range(0,len(outcome),8)]
        ascii_vals = [chr(int(b,base=2)) for b in ascii_bytes]
        print("".join(ascii_vals))
else:
    for f_outcome in outcomes:
        outcome = f_outcome[1:]
        print(outcome)

011100110110000101101101
sam

sam

sam


In [31]:
circuit.draw()

┌───┐                                                        ░ ┌───┐     »
 q_0: ┤ H ├──■────■────■────■────■────■────■────■────■────■────■───░─┤ X ├──■──»
      └───┘┌─┴─┐  │    │    │    │    │    │    │    │    │    │   ░ └───┘  │  »
 q_1: ─────┤ X ├──┼────┼────┼────┼────┼────┼────┼────┼────┼────┼───░────────┼──»
           └───┘  │    │    │    │    │    │    │    │    │    │   ░      ┌─┴─┐»
 q_2: ────────────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼───░──────┤ X ├»
                  │    │    │    │    │    │    │    │    │    │   ░      └───┘»
 q_3: ────────────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼───░───────────»
                  │    │    │    │    │    │    │    │    │    │   ░           »
 q_4: ────────────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼───░───────────»
                ┌─┴─┐  │    │    │    │    │    │    │    │    │   ░           »
 q_5: ──────────┤ X ├──┼────┼────┼────┼────┼────┼────┼────┼────┼───░───────────»
                └───┘┌─┴─┐  │    │    │    │    │    │    │    │   ░           »
 q_6: ───────────────┤ X ├──┼────┼────┼────┼────┼────┼────┼────┼───░───────────»
                     └───┘  │    │    │    │    │    │    │    │   ░           »
 q_7: ──────────────────────┼────┼────┼────┼────┼────┼────┼────┼───░───────────»
                            │    │    │    │    │    │    │    │   ░           »
 q_8: ──────────────────────┼────┼────┼────┼────┼────┼────┼────┼───░───────────»
                          ┌─┴─┐  │    │    │    │    │    │    │   ░           »
 q_9: ────────────────────┤ X ├──┼────┼────┼────┼────┼────┼────┼───░───────────»
                          └───┘  │    │    │    │    │    │    │   ░           »
q_10: ───────────────────────────┼────┼────┼────┼────┼────┼────┼───░───────────»
                                 │    │    │    │    │    │    │   ░           »
q_11: ───────────────────────────┼────┼────┼────┼────┼────┼────┼───░───────────»
                               ┌─┴─┐  │    │    │    │    │    │   ░           »
q_12: ─────────────────────────┤ X ├──┼────┼────┼────┼────┼────┼───░───────────»
                               └───┘┌─┴─┐  │    │    │    │    │   ░           »
q_13: ──────────────────────────────┤ X ├──┼────┼────┼────┼────┼───░───────────»
                                    └───┘┌─┴─┐  │    │    │    │   ░           »
q_14: ───────────────────────────────────┤ X ├──┼────┼────┼────┼───░───────────»
                                         └───┘┌─┴─┐  │    │    │   ░           »
q_15: ────────────────────────────────────────┤ X ├──┼────┼────┼───░───────────»
                                              └───┘  │    │    │   ░           »
q_16: ───────────────────────────────────────────────┼────┼────┼───░───────────»
                                                   ┌─┴─┐  │    │   ░           »
q_17: ─────────────────────────────────────────────┤ X ├──┼────┼───░───────────»
                                                   └───┘  │    │   ░           »
q_18: ────────────────────────────────────────────────────┼────┼───░───────────»
                                                          │    │   ░           »
q_19: ────────────────────────────────────────────────────┼────┼───░───────────»
                                                        ┌─┴─┐  │   ░           »
q_20: ──────────────────────────────────────────────────┤ X ├──┼───░───────────»
                                                        └───┘  │   ░           »
q_21: ─────────────────────────────────────────────────────────┼───░───────────»
                                                               │   ░           »
q_22: ─────────────────────────────────────────────────────────┼───░───────────»
                                                             ┌─┴─┐ ░           »
q_23: ───────────────────────────────────────────────────────┤ X ├─░───────────»
                                                             └───┘ ░           »
q_24: ───────────────────────────────